In [102]:
from sqlalchemy import Table, Column, DateTime, Float, Integer, PrimaryKeyConstraint, MetaData, create_engine 

# Datenbank erstellen/verbinden
engine = create_engine('sqlite:///data.db')
metadata = MetaData()

# Tabelle definieren
aggregate = Table(
    'dae', metadata,
    Column('timestamp', DateTime, nullable=False),
    # Column('aggregate', Float, nullable=False),
    Column('washingmachine', Float, nullable=False),
    Column('fridge', Float, nullable=False),
    Column('microwave', Float, nullable=False),
    Column('kettle', Float, nullable=False),
    Column('user_id', Integer, nullable=False),

    PrimaryKeyConstraint('timestamp', 'user_id')
)

# Tabellen erstellen
metadata.create_all(engine)


In [98]:
query = f"""
            --DROP TABLE seq2point
            --DROP TABLE seq2seq
            --DROP TABLE aggregate
            DROP TABLE dae
            """
engine.execute(query)

In [103]:
import pandas as pd

query = f"""
            SELECT *
            FROM dae
            """
user_id = 1
# query = f"""
#         SELECT timestamp, aggregate
#         FROM aggregate a
#         join (select timestamp,washingmachine,microwave,kettle 
#                 from seq2point) sp 
#                 on a.timestamp = sp.timestamp
#         WHERE user_id = {user_id}
#         """

df = pd.read_sql_query(query, engine)
df

,timestamp,washingmachine,fridge,microwave,kettle,user_id


In [19]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

# Erstellen der Verbindung zur SQLite-Datenbank
engine = create_engine('sqlite:///data.db')

# Erstellen der Session
Session = sessionmaker(bind=engine)
session = Session()

# Metadaten-Objekt erstellen
metadata = MetaData()

# Tabellennamen laden
table_names = engine.table_names()

# Daten aus jeder Tabelle abrufen
for table_name in table_names:
    table = Table(table_name, metadata, autoload_with=engine)
    query = table.select()
    results = session.execute(query).fetchall()
    print(f"Data from table {table_name}:")
    for result in results:
        # print(result)
        pass

# Schließen der Session
session.close()

C:\Users\stefa\AppData\Local\Temp\ipykernel_10484\1861582393.py:15: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


Data from table aggregate:
Data from table dae:
Data from table powerconsumption:
Data from table seq2point:
Data from table seq2seq:


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
user_id = 1
appliance = "microwave"
engine = create_engine('sqlite:///data.db')

def update_dataDB(df,engine,user_id):
    # vorhandene Daten auslesen
    df_existing = pd.read_sql('powerconsumption', con=engine)

    # angemeldeter Benutzer
    df_existing_update = df_existing[df_existing['user_id'] == user_id]
    # andere Benutzer
    df_existing_other = df_existing[df_existing['user_id'] != user_id]

    # Daten vom angemeldeten Benutzer zusammenführen -> neuen Daten haben Vorrang
    df_combined = pd.concat([df_existing_update, df])#.drop_duplicates(subset='timestamp', keep='last')

    # andere Daten wieder hinzufügen
    df_combined = df_combined.sort_values('timestamp').drop_duplicates(subset=['timestamp', 'user_id'], keep='last')
    df_combined = pd.concat([df_combined, df_existing_other]).sort_values('timestamp')

    # Daten in der DB aktualisieren
    df_combined.to_sql('powerconsumption', con=engine, if_exists='replace', index=True)

def predict_seq2seq(read_csv, write_prediction,db_engine,user_id):

    data = pd.read_csv(read_csv,header=None).rename(columns={0:'timestamp',1:'aggregate'})
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.set_index('timestamp')

    aggregate = data['aggregate']
    mean_agg, std_agg, aggregate = normalise(aggregate)

    aggregate = np.pad(aggregate, (WINDOW_SIZE//2, WINDOW_SIZE//2 +1))
    aggregate = np.array([aggregate[i:i+WINDOW_SIZE] for i in range(len(aggregate)-WINDOW_SIZE)])
    aggregate = np.expand_dims(aggregate, axis=-1)


    model = return_seq2point()
    for appliance in SUPPORTED_APPLIANCES.keys():
        WEIGHTS_PATH = os.path.join(BASEDIR,"weights",SUPPORTED_APPLIANCES[appliance]['seq2point_weights'])
        model.load_weights(WEIGHTS_PATH)
        predict = model.predict(aggregate)

        data[appliance] = np.squeeze(predict)
    
    data['user_id'] = user_id


    data.to_csv(write_prediction)
    update_dataDB(data,db_engine,user_id)

    return True

In [44]:
import pandas as pd

#     Column('washingmachine', Float, nullable=False),
#     Column('fridge', Float, nullable=False),
#     Column('microwave', Float, nullable=False),
#     Column('kettle', Float, nullable=False),

query = f"""
            SELECT *
            FROM seq2point
            """
user_id = 1
# query = f"""
#         SELECT timestamp, aggregate
#         FROM aggregate a
#         join (select timestamp,washingmachine,microwave,kettle 
#                 from seq2point) sp 
#                 on a.timestamp = sp.timestamp
#         WHERE user_id = {user_id}
#         """


df = pd.read_sql_query(query, engine)
df

# df['timestamp'] = pd.to_datetime(df['timestamp'])
# df = df.set_index('timestamp')
# hourly_sum = df[appliance].resample('H').sum()
# hourly_sum.index = hourly_sum.index.strftime('%Y-%m-%d %H:%M')

# labels = hourly_sum.index.to_list()
# values = hourly_sum.values.tolist()
# mean = round(hourly_sum.values.mean()/1000,2)
# bills = round(hourly_sum.values.sum()*0.008/12,2)

,timestamp,washingmachine,fridge,microwave,kettle,user_id


app.db auslesen


In [64]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

# Erstellen der Verbindung zur SQLite-Datenbank
engine = create_engine('sqlite:///app.db')

# Erstellen der Session
Session = sessionmaker(bind=engine)
session = Session()

# Metadaten-Objekt erstellen
metadata = MetaData()

# Tabellennamen laden
table_names = engine.table_names()

# Daten aus jeder Tabelle abrufen
for table_name in table_names:
    table = Table(table_name, metadata, autoload_with=engine)
    query = table.select()
    results = session.execute(query).fetchall()
    print(f"Data from table {table_name}:")
    for result in results:
        print(result)

# Schließen der Session
session.close()


Data from table ab_permission:
(1, 'can_this_form_post')
(2, 'can_this_form_get')
(3, 'can_list')
(4, 'can_show')
(5, 'can_add')
(6, 'can_download')
(7, 'can_edit')
(8, 'can_delete')
(9, 'can_userinfo')
(10, 'resetmypassword')
(11, 'resetpasswords')
(12, 'userinfoedit')
(13, 'menu_access')
(14, 'copyrole')
(15, 'can_chart')
(16, 'can_get')
(17, 'can_method1')
(18, 'can_method2')
(19, 'can_home')
(20, 'can_logout')
(21, 'can_appliance')
(22, 'can_history')
(23, 'can_dashboard')
Data from table ab_permission_view:
(1, 1, 5)
(2, 2, 5)
(3, 1, 6)
(4, 2, 6)
(5, 1, 7)
(6, 2, 7)
(7, 3, 9)
(8, 4, 9)
(9, 5, 9)
(10, 6, 9)
(11, 7, 9)
(12, 8, 9)
(13, 9, 9)
(14, 10, 9)
(15, 11, 9)
(16, 12, 9)
(17, 13, 10)
(18, 13, 11)
(19, 3, 12)
(20, 4, 12)
(21, 5, 12)
(22, 6, 12)
(23, 7, 12)
(24, 8, 12)
(25, 14, 12)
(26, 13, 13)
(27, 15, 14)
(28, 13, 15)
(29, 3, 16)
(30, 13, 17)
(31, 3, 18)
(32, 13, 19)
(33, 3, 20)
(34, 13, 21)
(35, 16, 22)
(36, 13, 24)
(37, 13, 25)
(38, 13, 26)
(39, 17, 23)
(40, 18, 23)
(41, 19, 

C:\Users\stefa\AppData\Local\Temp\ipykernel_10484\2455109047.py:15: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


In [87]:
id = 4
query = f'''select first_name, last_name from ab_user where id = {id}'''
name = engine.execute(query).fetchall()[0]
f'{name[0]} {name[1]}'

'Stefan Dimnik'